# Navigation MDP [1]

In [ ]:
# Other Imports.
from simple_rl.tasks import NavigationMDP
from simple_rl.agents import QLearningAgent
from simple_rl.planning import ValueIteration
from simple_rl.tasks.grid_world.GridWorldStateClass import GridWorldState

# Python Imports.
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
np.random.seed(0)
nvmdp = NavigationMDP(width=30, height=30, 
                      living_cell_types=['white', 'yellow', 'red', 'lime', 'magenta'],
                      living_cell_rewards=[0, 0, -10, -10, -10],
                      living_cell_distribution="probability",
                      living_cell_type_probs=[0.68, 0.17, 0.05, 0.05, 0.05],
                      goal_cell_locs=[(21,21)],
                      goal_cell_rewards=[1.],
                      goal_cell_types=["blue"],
                      slip_prob=0.00, step_cost=0.0, gamma=.99)
# Use "init_states" to request specific init states while sampling trajectories
# if "n_trajectory" is greater than # of init_states, remaining init states will be sampled randomly
state_trajectories, action_trajectories = nvmdp.sample_data(n_trajectory=8, 
                                                            init_states=[GridWorldState(2,2)], 
                                                            init_repetition=False)
nvmdp.visualize_grid(trajectories=state_trajectories, show_colorbar=True, show_rewards_colorbar=True, goal_marker="*c")

## Features used for short horizon

In [ ]:
print("Sample State Trajectory \nFeatures: Cell Indicator {} + Distsances to {} + Distance to {}\n".format(nvmdp.cell_types, nvmdp.cell_types, nvmdp.goal_cell_types))
[nvmdp.feature_at_state(s,
                        feature_type="indicator",
                        incl_cell_distances=True,
                        incl_goal_indicator=False,
                        incl_goal_distances=True,
                        normalize_distance=False, dtype=np.float).tolist() for s in state_trajectories[0]]

## Features used for long horizon

In [ ]:
print("Sample State Trajectory \nFeatures: Cell Indicator {} + Distance to {}\n".format(nvmdp.cell_types, nvmdp.goal_cell_types))
[nvmdp.feature_at_state(s,
                        feature_type="indicator",
                        incl_cell_distances=False,
                        incl_goal_indicator=False,
                        incl_goal_distances=True,
                        normalize_distance=False, dtype=np.float).tolist() for s in state_trajectories[0]]

## Multiple Goals

In [ ]:
np.random.seed(0)
nvmdp = NavigationMDP(width=30, height=30, 
                      living_cell_types=['white', 'yellow', 'red', 'lime', 'magenta'],
                      living_cell_rewards=[0, 0, -10, -10, -10],
                      living_cell_distribution="probability",
                      living_cell_type_probs=[0.68, 0.17, 0.05, 0.05, 0.05],
                      goal_cell_locs=[(21,21), (11,11)],
                      goal_cell_rewards=[1., 1.5],
                      goal_cell_types=["orange","blue"],
                      slip_prob=0.00, step_cost=0.0, gamma=.95)
state_trajectories, action_trajectories = nvmdp.sample_data(n_trajectory=8, 
                                                            init_states=[GridWorldState(2,2)], 
                                                            init_repetition=False)
nvmdp.visualize_grid(trajectories=state_trajectories, show_colorbar=True, show_rewards_colorbar=True, goal_marker="*c")
## Features: Cell Type Ind + Cell Type Dist + Goal Dist
print("Sample State Trajectory")
[nvmdp.feature_at_state(s,
                        feature_type="indicator",
                        incl_cell_distances=False,
                        incl_goal_indicator=False,
                        incl_goal_distances=True,
                        normalize_distance=False, dtype=np.float).tolist() for s in state_trajectories[0]]

## Custom Grid

In [ ]:
nvmdp = NavigationMDP(width=7, height=7,
                      living_cell_types=['white', 'yellow', 'red'],
                      living_cell_rewards=[0, 0, -10],
                      living_cell_distribution="manual",
                      living_cell_locs=[np.inf, np.inf, [(2,i) for i in range(1,7)]],
                      goal_cell_locs=[(7,1),(1,1)],
                      goal_cell_types=["blue", "orange"],
                      goal_cell_rewards=[1.,10.],
                      traj_init_cell_types=[0,1],
                      slip_prob=0.00, step_cost=0.0, gamma=.50)
state_trajectories, action_trajectories = nvmdp.sample_data(n_trajectory=41, init_states=[GridWorldState(1,1)], init_repetition=False)
nvmdp.visualize_grid(trajectories=state_trajectories, show_colorbar=True, show_rewards_colorbar=True, goal_marker="*c")

## Updating Goals dynamically

In [ ]:
nvmdp._reset_goals([(7,1),(1,1)],[10.,1.],["blue", "orange"])
state_trajectories, action_trajectories = nvmdp.sample_data(n_trajectory=41, init_states=[GridWorldState(1,1)], init_repetition=False)
nvmdp.visualize_grid(trajectories=state_trajectories, show_colorbar=True, show_rewards_colorbar=True, goal_marker="*c")

In [ ]:
nvmdp._reset_goals([(7,1),(1,1),(7,7)],[10.,1.,10.],["blue", "orange","purple"])
state_trajectories, action_trajectories = nvmdp.sample_data(n_trajectory=41, init_states=[GridWorldState(1,1)], init_repetition=False)
nvmdp.visualize_grid(trajectories=state_trajectories, show_colorbar=True, show_rewards_colorbar=True, goal_marker="*c")

In [ ]:
nvmdp._reset_goals([(7,1),(1,1),(7,7),(4,4)],[10.,1.,10.,5.],list(range(4))) 
state_trajectories, action_trajectories = nvmdp.sample_data(n_trajectory=41, init_states=[GridWorldState(1,1)], init_repetition=False)
plt.figure(figsize=(14,8))
nvmdp.visualize_grid(trajectories=state_trajectories, show_colorbar=True, show_rewards_colorbar=True, goal_marker="*c", new_fig=False, subplot_str="121")
nvmdp.visualize_grid(nvmdp.get_value_grid(), trajectories=state_trajectories, show_colorbar=True, show_rewards_colorbar=False, goal_marker="*c", new_fig=False, subplot_str="122", 
                     state_space_cmap=False, title="Value")

## Updating Rewards dynamically

In [ ]:
nvmdp.living_cell_rewards, nvmdp.goal_cell_rewards

In [ ]:
nvmdp._reset_rewards([0, 0, -0.01], [10.0, 1.0, 10.0, 5.0])
state_trajectories, action_trajectories = nvmdp.sample_data(n_trajectory=41, init_states=[GridWorldState(1,1)], init_repetition=False)
plt.figure(figsize=(14,8))
nvmdp.visualize_grid(trajectories=state_trajectories, show_colorbar=True, show_rewards_colorbar=True, goal_marker="*c", new_fig=False, subplot_str="121")
nvmdp.visualize_grid(nvmdp.get_value_grid(), trajectories=state_trajectories, show_colorbar=True, show_rewards_colorbar=False, goal_marker="*c", new_fig=False, subplot_str="122", 
                     state_space_cmap=False, title="Value")

## Reference

[1] MacGlashan, James, and Michael L. Littman. "Between Imitation and Intention Learning." IJCAI. 2015.